# SciTeX Context Module Tutorial

This notebook demonstrates the context managers in SciTeX, particularly for managing output suppression in scientific computing workflows.

## 1. Setup and Imports

In [ ]:
import scitex as stx
import numpy as np
import sys
import warnings
import time
from contextlib import redirect_stdout, redirect_stderr
import io

## 2. Output Suppression Basics

The `suppress_output` context manager (also available as `quiet`) allows you to selectively suppress stdout and stderr output.

### 2.1 Basic Usage

In [ ]:
# Normal output
print("This message will be displayed")

# Suppressed output
with stx.context.suppress_output():
    print("This message will NOT be displayed")
    print("Neither will this one")
    
print("This message will be displayed again")

### 2.2 Using the quiet Alias

In [ ]:
# The 'quiet' alias provides the same functionality
print("Before quiet block")

with stx.context.quiet():
    print("This is suppressed")
    for i in range(100):
        print(f"Iteration {i}")  # All suppressed
        
print("After quiet block")

### 2.3 Conditional Suppression

In [ ]:
# You can control suppression with a parameter
verbose = False

print("Running computation...")

with stx.context.suppress_output(suppress=not verbose):
    print("Debug: Starting calculation")
    result = sum(range(1000))
    print(f"Debug: Result = {result}")
    print("Debug: Calculation complete")

print(f"Final result: {result}")

# Now with verbose=True
print("\nRunning with verbose=True:")
verbose = True

with stx.context.suppress_output(suppress=not verbose):
    print("Debug: Starting calculation")
    result = sum(range(1000))
    print(f"Debug: Result = {result}")
    print("Debug: Calculation complete")

## 3. Practical Applications

### 3.1 Suppressing Library Warnings and Output

In [ ]:
# Many libraries produce verbose output during initialization or computation
def noisy_function():
    """Simulates a function that produces unwanted output."""
    print("Initializing...")
    print("Loading configuration...")
    print("Setting up parameters...")
    warnings.warn("This is a deprecation warning")
    print("Processing...")
    result = np.random.randn(100).mean()
    print("Cleaning up...")
    return result

# Without suppression
print("=== Without suppression ===")
result1 = noisy_function()
print(f"Result: {result1:.4f}\n")

# With suppression
print("=== With suppression ===")
with stx.context.quiet():
    result2 = noisy_function()
print(f"Result: {result2:.4f}")

### 3.2 Benchmarking with Clean Output

In [ ]:
def benchmark_function(func, *args, n_runs=100, suppress_output=True, **kwargs):
    """Benchmark a function with optional output suppression."""
    times = []
    
    for i in range(n_runs):
        start = time.time()
        
        with stx.context.suppress_output(suppress=suppress_output):
            result = func(*args, **kwargs)
            
        elapsed = time.time() - start
        times.append(elapsed)
        
        # Show progress every 10 runs
        if (i + 1) % 10 == 0:
            print(f"\rProgress: {i+1}/{n_runs}", end='', flush=True)
    
    print()  # New line after progress
    return np.array(times), result

# Example: Benchmark a noisy computation
def noisy_computation(size=1000):
    """A computation that prints progress."""
    data = np.random.randn(size, size)
    print(f"Processing {size}x{size} matrix...")
    
    for i in range(5):
        print(f"  Step {i+1}/5: Computing...")
        data = data @ data.T
        data = data / np.linalg.norm(data)
    
    print("Computation complete!")
    return data.sum()

# Benchmark with output suppression
print("Benchmarking with output suppression:")
times_quiet, _ = benchmark_function(noisy_computation, size=100, n_runs=20)

print(f"\nExecution time: {times_quiet.mean():.4f} ± {times_quiet.std():.4f} seconds")
print(f"Min: {times_quiet.min():.4f}s, Max: {times_quiet.max():.4f}s")

# Compare with one run showing output
print("\n\nOne run with output enabled:")
with stx.context.suppress_output(suppress=False):
    _ = noisy_computation(size=100)

### 3.3 Capturing Output Instead of Suppressing

In [ ]:
# Sometimes you want to capture output rather than suppress it
def capture_output(func, *args, **kwargs):
    """Capture stdout and stderr from a function."""
    stdout_capture = io.StringIO()
    stderr_capture = io.StringIO()
    
    with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
        result = func(*args, **kwargs)
    
    return result, stdout_capture.getvalue(), stderr_capture.getvalue()

# Example function with both stdout and stderr
def mixed_output_function():
    print("Normal output to stdout")
    print("More stdout output")
    print("Error message", file=sys.stderr)
    warnings.warn("A warning message")
    return 42

# Capture all output
result, stdout, stderr = capture_output(mixed_output_function)

print("=== Captured Output ===")
print(f"Result: {result}")
print(f"\nStdout ({len(stdout)} chars):")
print(stdout)
print(f"\nStderr ({len(stderr)} chars):")
print(stderr)

## 4. Advanced Usage Patterns

### 4.1 Nested Context Management

In [ ]:
# Context managers can be nested
def outer_function():
    print("Outer: Start")
    
    with stx.context.quiet():
        print("Outer: This is suppressed")
        inner_function()
        
    print("Outer: End")

def inner_function():
    print("Inner: This is also suppressed")
    
    # You can temporarily unsuppress within a suppressed context
    # by capturing and manually printing
    message = "Inner: Important message"
    # This would need to be handled differently to show
    
outer_function()

### 4.2 Creating a Verbose Mode Decorator

In [ ]:
from functools import wraps

def with_verbose_control(verbose_param='verbose'):
    """Decorator that adds verbose control to functions."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Get verbose flag from kwargs
            verbose = kwargs.get(verbose_param, False)
            
            # Remove verbose from kwargs if present
            kwargs_clean = {k: v for k, v in kwargs.items() if k != verbose_param}
            
            # Run function with output control
            with stx.context.suppress_output(suppress=not verbose):
                return func(*args, **kwargs_clean)
        
        return wrapper
    return decorator

# Example usage
@with_verbose_control()
def process_data(data):
    """Process data with optional verbose output."""
    print("Starting data processing...")
    print(f"Data shape: {data.shape}")
    
    # Simulate processing steps
    for i in range(5):
        print(f"Processing step {i+1}/5")
        data = data * 1.1 + np.random.randn(*data.shape) * 0.01
    
    print("Processing complete!")
    return data.mean()

# Test the decorated function
data = np.random.randn(100, 100)

print("With verbose=False (default):")
result1 = process_data(data)
print(f"Result: {result1:.4f}")

print("\nWith verbose=True:")
result2 = process_data(data, verbose=True)
print(f"Result: {result2:.4f}")

### 4.3 Progress Monitoring with Selective Output

In [ ]:
class ProgressMonitor:
    """A progress monitor that can selectively show output."""
    
    def __init__(self, total, desc="Progress", verbose=True, update_freq=0.1):
        self.total = total
        self.desc = desc
        self.verbose = verbose
        self.update_freq = update_freq
        self.current = 0
        self.last_update = 0
        self.start_time = time.time()
        
    def update(self, n=1):
        """Update progress by n steps."""
        self.current += n
        
        # Only update display if enough progress made
        progress = self.current / self.total
        if progress - self.last_update >= self.update_freq or self.current >= self.total:
            if self.verbose:
                self._display()
            self.last_update = progress
    
    def _display(self):
        """Display progress bar."""
        progress = self.current / self.total
        bar_length = 40
        filled = int(bar_length * progress)
        bar = '█' * filled + '░' * (bar_length - filled)
        
        elapsed = time.time() - self.start_time
        rate = self.current / elapsed if elapsed > 0 else 0
        
        print(f"\r{self.desc}: [{bar}] {progress*100:.1f}% "
              f"({self.current}/{self.total}) {rate:.1f} it/s", 
              end='', flush=True)
        
        if self.current >= self.total:
            print()  # New line when complete

# Example: Process with optional progress display
def process_with_progress(n_items, show_progress=True, show_details=False):
    """Process items with configurable output."""
    results = []
    
    progress = ProgressMonitor(n_items, "Processing", verbose=show_progress)
    
    for i in range(n_items):
        # Suppress detailed output unless requested
        with stx.context.suppress_output(suppress=not show_details):
            print(f"\nProcessing item {i+1}:")
            print(f"  - Loading data...")
            print(f"  - Applying transformation...")
            print(f"  - Validating results...")
            
        # Simulate work
        time.sleep(0.01)
        results.append(np.random.randn())
        
        progress.update()
    
    return results

# Test different output configurations
print("1. Progress only (no details):")
results1 = process_with_progress(50, show_progress=True, show_details=False)

print("\n2. No output at all:")
results2 = process_with_progress(50, show_progress=False, show_details=False)
print("Done!")

print("\n3. Full details (first 5 items):")
results3 = process_with_progress(5, show_progress=True, show_details=True)

## 5. Error Handling with Suppressed Output

In [ ]:
# Error messages are also suppressed, so be careful
def safe_suppress(func, *args, **kwargs):
    """Safely suppress output but capture errors."""
    error_buffer = io.StringIO()
    
    try:
        with stx.context.suppress_output():
            # Redirect stderr to our buffer to capture errors
            with redirect_stderr(error_buffer):
                result = func(*args, **kwargs)
        return result, None
    except Exception as e:
        # Return the error and any error output
        return None, (str(e), error_buffer.getvalue())

# Test functions
def good_function():
    print("Working...")
    return 42

def bad_function():
    print("Starting...")
    print("About to fail...", file=sys.stderr)
    raise ValueError("Something went wrong!")

# Test both functions
result1, error1 = safe_suppress(good_function)
print(f"Good function: result={result1}, error={error1}")

result2, error2 = safe_suppress(bad_function)
print(f"\nBad function: result={result2}")
if error2:
    print(f"Error message: {error2[0]}")
    print(f"Error output: {error2[1]}")

## 6. Integration with Logging

In [ ]:
import logging

# Configure logging
logger = logging.getLogger('example')
logger.setLevel(logging.DEBUG)

# Create handlers
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
log_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(log_format)

# Add handlers to the logger
logger.addHandler(console_handler)

def process_with_logging():
    """Example function that uses both print and logging."""
    print("This is a print statement")
    logger.info("This is an info log")
    logger.debug("This is a debug log (won't show)")
    logger.warning("This is a warning log")
    return "Done"

# Normal execution
print("=== Normal execution ===")
result = process_with_logging()

# With output suppression (note: logging is NOT suppressed by default)
print("\n=== With suppress_output ===")
with stx.context.suppress_output():
    result = process_with_logging()
    
print("\nNote: Logging output is not suppressed by suppress_output!")

## 7. Performance Considerations

In [ ]:
# Test performance impact of output suppression
def benchmark_output_suppression():
    """Compare performance with and without output suppression."""
    n_iterations = 10000
    
    # Function that produces output
    def chatty_function():
        print(f"Processing item...")
        return sum(range(100))
    
    # Benchmark without suppression (redirect to buffer to avoid spam)
    buffer = io.StringIO()
    start = time.time()
    with redirect_stdout(buffer):
        for _ in range(n_iterations):
            chatty_function()
    time_no_suppress = time.time() - start
    
    # Benchmark with suppression
    start = time.time()
    with stx.context.suppress_output():
        for _ in range(n_iterations):
            chatty_function()
    time_suppress = time.time() - start
    
    print(f"Performance comparison ({n_iterations} iterations):")
    print(f"Without suppression: {time_no_suppress:.3f}s")
    print(f"With suppression: {time_suppress:.3f}s")
    print(f"Suppression is {time_no_suppress/time_suppress:.2f}x faster")
    print(f"\nOutput size that would have been printed: {len(buffer.getvalue())/1024:.1f} KB")

benchmark_output_suppression()

## 8. Best Practices and Summary

### Best Practices

1. **Use for Clean Output**:
   ```python
   # Suppress verbose library initialization
   with stx.context.quiet():
       import some_verbose_library
   ```

2. **Conditional Verbosity**:
   ```python
   def compute(data, verbose=False):
       with stx.context.suppress_output(suppress=not verbose):
           # Detailed processing steps
           pass
   ```

3. **Error Safety**:
   ```python
   # Always handle errors when suppressing output
   try:
       with stx.context.quiet():
           result = risky_operation()
   except Exception as e:
       logger.error(f"Operation failed: {e}")
   ```

4. **Progress Reporting**:
   ```python
   # Show progress while suppressing details
   for i in range(n_items):
       with stx.context.quiet():
           process_item(i)
       print(f"\rProgress: {i+1}/{n_items}", end='')
   ```

### Summary

The context module provides:
- Clean output control for scientific computing
- Easy integration with existing code
- Performance benefits when suppressing large outputs
- Flexible conditional suppression

Use cases:
- Benchmarking without output noise
- Clean progress reporting
- Suppressing verbose library messages
- Creating user-friendly interfaces

In [ ]:
print("Context module tutorial completed!")
print("\nKey takeaways:")
print("1. Use suppress_output() or quiet() to control output")
print("2. Great for creating clean, professional outputs")
print("3. Can significantly improve performance when suppressing large outputs")
print("4. Remember to handle errors appropriately when suppressing output")